In [12]:
import torch
from resnet8 import ResNet8

model = ResNet8()
model.eval()

ResNet8(
  (stem): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (first_stack): ResNetBlock(
    (block): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (residual): Identity()
  )
  (second_stack): ResNetBlock(
    (block): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), p

In [10]:
for n, m in model.named_modules():
  print(n, m)
    # if isinstance(m, torch.nn.Conv2d):
    #     print(n, m.weight.shape)
    #     print(n, m.bias.shape)
    # if isinstance(m, torch.nn.BatchNorm2d):
    #     print(n, m.weight.shape)
    #     print(n, m.bias.shape)
    # if isinstance(m, torch.nn.Linear):
    #     print(n, m.weight.shape)
    #     print(n, m.bias.shape)

 ResNet8(
  (stem): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (first_stack): ResNetBlock(
    (block): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (residual): Identity()
  )
  (second_stack): ResNetBlock(
    (block): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), 

In [17]:
model_fused = torch.quantization.fuse_modules(model, 
    [
        ['stem.0', 'stem.1', 'stem.2'],
        ['first_stack.block.0', 'first_stack.block.1', 'first_stack.block.2'],
        ['first_stack.block.3', 'first_stack.block.4'],
        ['second_stack.block.0', 'second_stack.block.1', 'second_stack.block.2'],
        ['second_stack.block.3', 'second_stack.block.4'],
        ['third_stack.block.0', 'third_stack.block.1', 'third_stack.block.2'],
        ['third_stack.block.3', 'third_stack.block.4']
    ])

# print(mode_fused)

In [18]:
from torch.utils.mobile_optimizer import optimize_for_mobile


def prepare_save(model, fused):
    model.qconfig = torch.quantization.get_default_qconfig('qnnpack')
    torch.quantization.prepare(model, inplace=True)
    torch.quantization.convert(model, inplace=True)
    torchscript_model = torch.jit.script(model)
    torchscript_model_optimized = optimize_for_mobile(torchscript_model)
    torch.jit.save(torchscript_model_optimized, "model.pt" if not fused else "model_fused.pt")

prepare_save(model_fused, True)

/home/scott306lr/conda_envs/pytorch/lib/python3.10/site-packages/torch/ao/quantization/observer.py:1207: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(
